# Vector Database

## Goal

In this notebook, we will explore the vector database `Qdrant`. We will index and query the documents.

In [1]:
from qdrant_client import QdrantClient

In [2]:
client = QdrantClient(":memory:")
# or
# client = QdrantClient(path="tmp/tmp.db")
# client.close()

In [3]:
cat_facts = [
    "Cats have 32 muscles in each ear, allowing for their unique ability to rotate their ears independently.",
    "A group of cats is called a 'clowder'.",
    "Cats can't taste sweetness.",
    "A cat's whiskers help them navigate in the dark.",
    "Cats have three eyelids: upper, lower, and third (also known as nictitating membrane).",
    "Cats spend 1/3 of their waking hours sleeping.",
    "A cat's purr can be a sign of happiness or self-soothing.",
    "Cats have unique nose prints, just like humans have fingerprints.",
    "The world's largest domesticated cat was Muffin, who measured 48.5 inches long and weighed 46.3 pounds.",
    "Cats can't see in complete darkness but their night vision is excellent due to a reflective layer in the back of their eyes called the tapetum lucidum.",
]
dog_facts = [
    "A dog's sense of smell is up to 10,000 times more sensitive than a human's.",
    "Dogs can hear sounds at frequencies as high as 40,000 Hz, while humans can only hear up to 20,000 Hz.",
    "A group of dogs is called a 'pack'.",
    "Dogs have three eyelids: upper, lower, and nictitating membrane (third eyelid).",
    "Dogs can dream just like humans do, and their brain waves show similar patterns during sleep.",
    "The world's smallest dog breed is the Chihuahua, with adults weighing as little as 2 pounds.",
    "A dog's tail language is more complex than human body language, conveying emotions and intentions.",
    "Dogs can be right- or left-pawed, just like humans are right- or left-handed.",
    "The oldest known dog remains dates back to around 14,000 years ago, during the Late Pleistocene era.",
    "Dogs have a unique nose print, just like humans have fingerprints.",
]

## Preparing Metadata

In [4]:
docs = cat_facts + dog_facts
metadata = [{"source": "cats"}] * len(cat_facts) + [{"source": "dogs"}] * len(dog_facts)
ids = list(range(len(docs)))

## Adding documents to collection

Below, we add the documents together with the metadatas and ids to our vector database.
The default embedding is used if we did not specify them

In [7]:
client.add(
    collection_name="cats_and_dogs",
    documents=docs,
    metadata=metadata,
    ids=ids,
)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [10]:
# Default embedding used.
print(client.embedding_model_name)

BAAI/bge-small-en


## Querying documents

In [6]:
search_result = client.query(
    collection_name="cats_and_dogs", query_text="what is the cat whisker for?"
)
search_result

[QueryResponse(id=3, embedding=None, sparse_embedding=None, metadata={'document': "A cat's whiskers help them navigate in the dark.", 'source': 'cats'}, document="A cat's whiskers help them navigate in the dark.", score=0.8960122317479164),
 QueryResponse(id=1, embedding=None, sparse_embedding=None, metadata={'document': "A group of cats is called a 'clowder'.", 'source': 'cats'}, document="A group of cats is called a 'clowder'.", score=0.8371795788861598),
 QueryResponse(id=6, embedding=None, sparse_embedding=None, metadata={'document': "A cat's purr can be a sign of happiness or self-soothing.", 'source': 'cats'}, document="A cat's purr can be a sign of happiness or self-soothing.", score=0.8254998482294487),
 QueryResponse(id=4, embedding=None, sparse_embedding=None, metadata={'document': 'Cats have three eyelids: upper, lower, and third (also known as nictitating membrane).', 'source': 'cats'}, document='Cats have three eyelids: upper, lower, and third (also known as nictitating me

## Narrowing search with filters

In [11]:
from qdrant_client.models import FieldCondition, Filter, MatchValue

search_result = client.query(
    collection_name="cats_and_dogs",
    query_text="what is the cat whisker for?",
    query_filter=Filter(
        must=[FieldCondition(key="source", match=MatchValue(value="cats"))]
    ),
)

for res in search_result:
    print(dict(doc=res.document, score=res.score))

{'doc': "A cat's whiskers help them navigate in the dark.", 'score': 0.8960122052835192}
{'doc': "A group of cats is called a 'clowder'.", 'score': 0.8371795647410071}
{'doc': "A cat's purr can be a sign of happiness or self-soothing.", 'score': 0.8254998513657924}
{'doc': 'Cats have three eyelids: upper, lower, and third (also known as nictitating membrane).', 'score': 0.8023456208085149}
{'doc': 'Cats have unique nose prints, just like humans have fingerprints.', 'score': 0.7981833555293472}
{'doc': 'Cats have 32 muscles in each ear, allowing for their unique ability to rotate their ears independently.', 'score': 0.7954061802722565}
{'doc': "Cats can't taste sweetness.", 'score': 0.7869827666131266}
{'doc': 'Cats spend 1/3 of their waking hours sleeping.', 'score': 0.780144426020518}
{'doc': "Cats can't see in complete darkness but their night vision is excellent due to a reflective layer in the back of their eyes called the tapetum lucidum.", 'score': 0.7533777384436582}
{'doc': "Th